In this example, we order and process imagery with the [image preprocessor](http://gbdxdocs.digitalglobe.com/docs/advanced-image-preprocessor). Each image is specified by a catalog id. 
Imagery for a given location can be discovered at [discover.digitalglobe.com](https://discover.digitalglobe.com/).

In [1]:
import os
os.environ['GBDX_USERNAME'] = ''
os.environ['GBDX_PASSWORD'] = ''
os.environ['GBDX_CLIENT_ID'] = '' 
os.environ['GBDX_CLIENT_SECRET'] = ''

import gbdxtools

gbdx = gbdxtools.Interface()

For the same image, we will generate the:

+ orthorectified, atmospherically compensated multispectral image;
+ the orthorectified, panchromatic image;
+ the orthorectified, atmospherically compensated pansharpened image with default Dynamic Range Adjustment (DRA);
+ and the orthorectified atmospherically compensated pansharpened image with base layer matching. 

It is sometimes the case that pansharpening an image with the default DRA of the image preprocessor results in poor viewing quality. This is the reason for using the base layer matching task. In addition, note that all images are projected in UTM. (If the projection is not specified, the default projection is EPSG:4326.)

In [ ]:
catid = '104001001D114300'

order = gbdx.Task('Auto_Ordering', cat_id=catid)
order.impersonation_allowed = True

# multispectral 
aop1 = gbdx.Task('AOP_Strip_Processor',
                 data=order.outputs.s3_location.value,
                 bands='MS',
                 enable_dra=False,
                 enable_pansharpen=False,
                 enable_acomp=True,
                 ortho_epsg='UTM')     

# panchromatic
aop2 = gbdx.Task('AOP_Strip_Processor',
                 data=order.outputs.s3_location.value,
                 bands='PAN',
                 enable_dra=False,
                 enable_pansharpen=False,
                 enable_acomp=False,
                 ortho_epsg='UTM')     

# pansharpened with default DRA
aop3 = gbdx.Task('AOP_Strip_Processor',
                 data=order.outputs.s3_location.value,                     
                 ortho_epsg='UTM')      

# pansharpened with baselayer matching
aop4 = gbdx.Task('AOP_Strip_Processor',
                 data=order.outputs.s3_location.value,
                 enable_dra=False,                     # disable automatic dra 
                 ortho_epsg='UTM')                         
blm = gbdx.Task('baselayermatch',
                data=aop3.outputs.data.value,
                cloud_id=catid)                        # take clouds into account when dra'ing

wf = gbdx.Workflow([order, aop1, aop2, aop3, aop4, blm])

from os.path import join
import uuid
random_str = str(uuid.uuid4())
output_location = join('platform-stories/trial-runs', random_str)

wf.savedata(aop1.outputs.data, join(output_location, 'ms', catid))
wf.savedata(aop2.outputs.data, join(output_location, 'pan', catid))
wf.savedata(aop3.outputs.data, join(output_location, 'ps', catid))
wf.savedata(blm.outputs.data, join(output_location, 'ps-blm', catid))

wf.execute()

In [7]:
print output_location
print wf.status

platform-stories/trial-runs/006a3648-03fa-45d7-b198-c2db162b9f1e
{u'state': u'running', u'event': u'started'}


If we don't want to save the entire image strip but only a part of it specified by a bounding box, we can chain the image preprocessor to a cropping task. Here is an example for another image.

In [ ]:
W, S, E, N = (-0.4977187514305114, 51.45510667726173, -0.42132943868637085, 51.48600965926799)

wkt = 'POLYGON (({} {}, {} {}, {} {}, {} {}, {} {}))'.format(W, N, E, N, E, S, W, S, W, N)
catid = '103001002E5B2600'

order = gbdx.Task('Auto_Ordering', cat_id=catid)
order.impersonation_allowed = True

# multispectral 
aop1 = gbdx.Task('AOP_Strip_Processor',
                 data=order.outputs.s3_location.value,
                 bands='MS',
                 enable_dra=False,
                 enable_pansharpen=False,
                 enable_acomp=True,
                 ortho_epsg='UTM')     

# panchromatic
aop2 = gbdx.Task('AOP_Strip_Processor',
                 data=order.outputs.s3_location.value,
                 bands='PAN',
                 enable_dra=False,
                 enable_pansharpen=False,
                 enable_acomp=False,
                 ortho_epsg='UTM')     

# pansharpened with default DRA
aop3 = gbdx.Task('AOP_Strip_Processor',
                 data=order.outputs.s3_location.value,                     
                 ortho_epsg='UTM')      

# pansharpened with baselayer matching
aop4 = gbdx.Task('AOP_Strip_Processor',
                 data=order.outputs.s3_location.value,
                 enable_dra=False,                     # disable automatic dra 
                 ortho_epsg='UTM')                         
blm = gbdx.Task('baselayermatch',
                data=aop4.outputs.data.value,
                cloud_id=catid)                        # take clouds into account when dra'ing

# cropping
crop1 = gbdx.Task('CropGeotiff', 
                  data=aop1.outputs.data.value,
                  wkt_0=wkt, wkt_1=wkt)                # we need to define two ports; a little bug in CropGeotiff
crop2 = gbdx.Task('CropGeotiff', 
                  data=aop2.outputs.data.value,
                  wkt_0=wkt, wkt_1=wkt) 
crop3 = gbdx.Task('CropGeotiff', 
                  data=aop3.outputs.data.value,
                  wkt_0=wkt, wkt_1=wkt) 
crop4 = gbdx.Task('CropGeotiff', 
                  data=blm.outputs.data.value,
                  wkt_0=wkt, wkt_1=wkt) 

wf = gbdx.Workflow([order, aop1, aop2, aop3, aop4, blm, crop1, crop2, crop3, crop4])

from os.path import join
import uuid
random_str = str(uuid.uuid4())
output_location = join('platform-stories/trial-runs', random_str)

wf.savedata(crop1.outputs.data_0, join(output_location, 'ms', catid))
wf.savedata(crop2.outputs.data_0, join(output_location, 'pan', catid))
wf.savedata(crop3.outputs.data_0, join(output_location, 'ps', catid))
wf.savedata(crop4.outputs.data_0, join(output_location, 'ps-blm', catid))

wf.execute()

In [8]:
print output_location
print wf.status

platform-stories/trial-runs/006a3648-03fa-45d7-b198-c2db162b9f1e
{u'state': u'running', u'event': u'started'}


If you want to process multiple images in parallel, all you need to do is loop over the corresponding catalog ids. GBDX will execute the corresponding workflows in parallel.